# Setup of repository

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
my_repo_location = '/content/drive/MyDrive/Colab Notebooks/w266/petl-athiruve-hanmaegeo-raulmy'
%cd /content/drive/MyDrive/Colab\ Notebooks/w266/petl-athiruve-hanmaegeo-raulmy

/content/drive/MyDrive/Colab Notebooks/w266/petl-athiruve-hanmaegeo-raulmy


In [5]:
import os
import sys
import shutil
import pickle
from pathlib import Path
base = Path(my_repo_location)
sys.path.insert(0, str(base))

In [6]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.5 MB/s eta 0:00:0

## Need to modify the __ init __.py to expose some classes that are not generally made available

### Backup the existing __ init __.py

In [23]:
import transformers
src_file = os.path.join(os.path.dirname(transformers.__file__), 'models/t5/__init__.py')
dst_file = os.path.join(os.path.dirname(transformers.__file__), 'models/t5/__init__backup.py ')
shutil.move(src_file, dst_file)\
# !mv /usr/local/lib/python3.9/dist-packages/transformers/models/t5/__init__.py /usr/local/lib/python3.9/dist-packages/transformers/models/t5/__init__backup.py

'/usr/local/lib/python3.9/dist-packages/transformers/models/t5/__init__backup.py '

### Replace with the one we want

In [24]:
# !cp utils/transformer.model.t5__init__.py /usr/local/lib/python3.9/dist-packages/transformers/models/t5/__init__.py
shutil.copy('utils/transformer.model.t5__init__.py', src_file)

'/usr/local/lib/python3.9/dist-packages/transformers/models/t5/__init__.py'

In [26]:
from utils import constants
from utils.train import run_benchmark

In [ ]:
def run_model(benchmark, model_config: dict, optimizer_lrs: dict, checkpoint_filepath: str, debug: bool = False,
              prefix='', batch_size=None):
    """

    Args:
         model_config: Dictionary:
                'model_checkpoint': <t5-small, t5-base>
                'which_model': 'fft' -> full fine-tuning of all parameters.
                               'soft' -> soft prompt is tuned
                               'library' -> library of soft prompts
                'epochs': <Optional>
        benchmark:
        checkpoint_filepath: Location of where the checkpoints are stored
        optimizer_lrs: optimizer learning rates for each dataset
        debug: if True then eager model of evaluation is run, else graph mode
        prefix: Prefix to add to the model names
        batch_size: Training batch size
    Returns:

    """

    cache_path = os.path.join(os.getcwd(), "cache")
    batch_size = 100 if batch_size is None else batch_size

    #  Run the superglue benchmnark
    run_benchmark(model_config=model_config, optimizer_lrs=optimizer_lrs, batch_size=batch_size,
                  cache_path=cache_path, checkpoint_filepath=checkpoint_filepath, debug=debug, benchmark=benchmark,
                  one_task=None, prefix=prefix)

## Get modelling conditions

In [ ]:
model_checkpoint_ = 't5-small'
which_model_ = 'fft'
benchmark_ = 'target'
batch_size_ = 100
model_config_ = {'model_checkpoint': model_checkpoint_, 'which_model': which_model_, 'epochs': 50}
checkpoint_filepath_ = os.path.join(os.getcwd(), "checkpoints")

## Make sure optimizer was run and learning rates are extracted

In [ ]:
# Benchmark of target signifies target tasks
# Learning rate on log scale
try:
    all_tasks_tag = model_checkpoint_ + '-' + which_model_ + '-' + benchmark_
    filepath = os.path.join(checkpoint_filepath_, 'optimizer/lro-' + all_tasks_tag + '.p')
    with open(filepath, 'rb') as infi:
        optimizer_lrs_ = pickle.load(infi)
except FileNotFoundError:
    raise FileNotFoundError('Was optimization run to get learning rates?')


# Scale the learning rate by the number of epochs * number of batches per epoch
optimizer_lrs_ = {k: v/(constants.COUNTS[k]/batch_size_)**0.5 for k, v in optimizer_lrs_.items()}

In [ ]:
# Benchmark can be given as this tuple of atsks or a benchmark name such as 'glue' or 'super_glue'
run_model(benchmark=benchmark_, model_config=model_config_, optimizer_lrs=optimizer_lrs_, debug=False,
          prefix='athiruve', batch_size=batch_size_, checkpoint_filepath=checkpoint_filepath_)